# End to End Industrial IoT (IIoT) on Azure Databricks
## Part 1: Data Engineering
This notebook demonstrates the following architecture for IIoT Ingest, Processing and Analytics on Azure. The following architecture is implemented for the demo. 
<img src="https://sguptasa.blob.core.windows.net/random/iiot_blog/end_to_end_architecture.png" width=800>

The notebook is broken into sections following these steps:
1. **Data Ingest** - stream real-time raw sensor data from Azure IoT Hubs into the Delta format in Azure Storage
2. **Data Processing** - stream process sensor data from raw (Bronze) to silver (aggregated) to gold (enriched) Delta tables on Azure Storage

In [0]:
# AzureML Workspace info (name, region, resource group and subscription ID) for model deployment
dbutils.widgets.text("Storage Account","<your ADLS Gen 2 account name>","Storage Account")

## Step 1 - Environment Setup

The pre-requisites are listed below:

### Azure Services Required
* Azure IoT Hub 
* [Azure IoT Simulator](https://azure-samples.github.io/raspberry-pi-web-simulator/) running with the code provided in [this github repo](https://github.com/tomatoTomahto/azure_databricks_iot/blob/master/azure_iot_simulator.js) and configured for your IoT Hub
* ADLS Gen 2 Storage account with a container called `iot`
* (Optional) Azure Synapse SQL Pool call `iot`

### Azure Databricks Configuration Required
* 3-node (min) Databricks Cluster running **DBR 7.0+** and the following libraries:
 * **Azure Event Hubs Connector for Databricks** - Maven coordinates `com.microsoft.azure:azure-eventhubs-spark_2.12:2.3.17`
* The following Secrets defined in scope `iot`
 * `iothub-cs` - Connection string for your IoT Hub **(Important - use the [Event Hub Compatible](https://devblogs.microsoft.com/iotdev/understand-different-connection-strings-in-azure-iot-hub/) connection string)**
 * `adls_key` - Access Key to ADLS storage account **(Important - use the [Access Key](https://raw.githubusercontent.com/tomatoTomahto/azure_databricks_iot/master/bricks.com/blog/2020/03/27/data-exfiltration-protection-with-azure-databricks.html))**
 * (Optional) `synapse_cs` - JDBC connect string to your Synapse SQL Pool **(Important - use the [SQL Authentication](https://docs.microsoft.com/en-us/azure/databricks/data/data-sources/azure/synapse-analytics#spark-driver-to-azure-synapse) with username/password connection string)**
* The following notebook widgets populated:
 * `Storage Account` - Name of your storage account

In [0]:
# Setup access to storage account for temp data when pushing to Synapse
storage_account = dbutils.widgets.get("Storage Account")
spark.conf.set(f"fs.azure.account.key.{storage_account}.dfs.core.windows.net", dbutils.secrets.get("iot","adls_key"))

# Setup storage locations for all data
ROOT_PATH = f"abfss://iot@{storage_account}.dfs.core.windows.net/"
BRONZE_PATH = ROOT_PATH + "bronze/"
SILVER_PATH = ROOT_PATH + "silver/"
GOLD_PATH = ROOT_PATH + "gold/"
SYNAPSE_PATH = ROOT_PATH + "synapse/"
CHECKPOINT_PATH = ROOT_PATH + "checkpoints/"

# Other initializations
IOT_CS = dbutils.secrets.get('iot','iothub-cs') # IoT Hub connection string (Event Hub Compatible)
ehConf = { 
  'eventhubs.connectionString':sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(IOT_CS),
  'ehName':dbutils.widgets.get("Event Hub Name")
}

# Enable auto compaction and optimized writes in Delta
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled","true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled","true")

# Pyspark and ML Imports
import os, json, requests
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType

In [0]:
# Make sure root path is empty
dbutils.fs.rm(ROOT_PATH, True)

Out[3]: True

In [0]:
%sql
-- Clean up tables & views
DROP TABLE IF EXISTS iot.turbine_raw;
DROP TABLE IF EXISTS iot.weather_raw;
DROP TABLE IF EXISTS iot.turbine_agg;
DROP TABLE IF EXISTS iot.weather_agg;
DROP TABLE IF EXISTS iot.turbine_enriched;
DROP TABLE IF EXISTS iot.turbine_power;
DROP TABLE IF EXISTS iot.turbine_maintenance;
DROP VIEW IF EXISTS iot.turbine_combined;
DROP VIEW IF EXISTS iot.feature_view;
DROP TABLE IF EXISTS iot.turbine_life_predictions;
DROP TABLE IF EXISTS iot.turbine_power_predictions;
DROP DATABASE IF EXISTS iot;
CREATE DATABASE IF NOT EXISTS iot;

## Step 2 - Data Ingest from IoT Hubs
Azure Databricks provides a native connector to IoT and Event Hubs. Below, we will use PySpark Structured Streaming to read from an IoT Hub stream of data and write the data in it's raw format directly into Delta. 

Make sure that your IoT Simulator is sending payloads to IoT Hub as shown below.

<img src="https://sguptasa.blob.core.windows.net/random/iiot_blog/iot_simulator.gif" width=800>

We have two separate types of data payloads in our IoT Hub:
1. **Turbine Sensor readings** - this payload contains `date`,`timestamp`,`deviceid`,`rpm` and `angle` fields
2. **Weather Sensor readings** - this payload contains `date`,`timestamp`,`temperature`,`humidity`,`windspeed`, and `winddirection` fields

We split out the two payloads into separate streams and write them both into Delta locations on Azure Storage. We are able to query these two Bronze tables *immediately* as the data streams in.

In [0]:
# Schema of incoming data from IoT hub
schema = "timestamp timestamp, deviceId string, temperature double, humidity double, windspeed double, winddirection string, rpm double, angle double"

# Read directly from IoT Hub using the EventHubs library for Databricks
iot_stream = (
  spark.readStream.format("eventhubs")                                               # Read from IoT Hubs directly
    .options(**ehConf)                                                               # Use the Event-Hub-enabled connect string
    .load()                                                                          # Load the data
    .withColumn('reading', F.from_json(F.col('body').cast('string'), schema))        # Extract the "body" payload from the messages
    .select('reading.*', F.to_date('reading.timestamp').alias('date'))               # Create a "date" field for partitioning
)

# Split our IoT Hub stream into separate streams and write them both into their own Delta locations
write_turbine_to_delta = (
  iot_stream.filter('temperature is null')                                           # Filter out turbine telemetry from other data streams
    .select('date','timestamp','deviceId','rpm','angle')                             # Extract the fields of interest
    .writeStream.format('delta')                                                     # Write our stream to the Delta format
    .partitionBy('date')                                                             # Partition our data by Date for performance
    .option("checkpointLocation", CHECKPOINT_PATH + "turbine_raw")                   # Checkpoint so we can restart streams gracefully
    .start(BRONZE_PATH + "turbine_raw")                                              # Stream the data into an ADLS Path
)

write_weather_to_delta = (
  iot_stream.filter(iot_stream.temperature.isNotNull())                              # Filter out weather telemetry only
    .select('date','deviceid','timestamp','temperature','humidity','windspeed','winddirection') 
    .writeStream.format('delta')                                                     # Write our stream to the Delta format
    .partitionBy('date')                                                             # Partition our data by Date for performance
    .option("checkpointLocation", CHECKPOINT_PATH + "weather_raw")                   # Checkpoint so we can restart streams gracefully
    .start(BRONZE_PATH + "weather_raw")                                              # Stream the data into an ADLS Path
)

# Create the external tables once data starts to stream in
while True:
  try:
    spark.sql(f'CREATE TABLE IF NOT EXISTS iot.turbine_raw USING DELTA LOCATION "{BRONZE_PATH + "turbine_raw"}"')
    spark.sql(f'CREATE TABLE IF NOT EXISTS iot.weather_raw USING DELTA LOCATION "{BRONZE_PATH + "weather_raw"}"')
    break
  except:
    pass

In [0]:
%sql 
-- We can query the data directly from storage immediately as soon as it starts streams into Delta 
SELECT * FROM iot.turbine_raw WHERE deviceid = 'WindTurbine-1'

date,timestamp,deviceId,rpm,angle
2021-03-05,2021-03-05T00:04:17.000+0000,WindTurbine-1,8.355236850129419,6.310832243863243
2021-03-05,2021-03-05T00:04:38.000+0000,WindTurbine-1,8.94695441847841,7.828585116168608


## Step 2 - Data Processing in Delta
While our raw sensor data is being streamed into Bronze Delta tables on Azure Storage, we can create streaming pipelines on this data that flow it through Silver and Gold data sets.

We will use the following schema for Silver and Gold data sets:

<img src="https://sguptasa.blob.core.windows.net/random/iiot_blog/iot_delta_bronze_to_gold.png" width=800>

### 2a. Delta Bronze (Raw) to Delta Silver (Aggregated)
The first step of our processing pipeline will clean and aggregate the measurements to 1 hour intervals. 

Since we are aggregating time-series values and there is a likelihood of late-arriving data and data changes, we will use the [**MERGE**](https://docs.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/language-manual/merge-into?toc=https%3A%2F%2Fdocs.microsoft.com%2Fen-us%2Fazure%2Fazure-databricks%2Ftoc.json&bc=https%3A%2F%2Fdocs.microsoft.com%2Fen-us%2Fazure%2Fbread%2Ftoc.json) functionality of Delta to upsert records into target tables. 

MERGE allows us to upsert source records into a target storage location. This is useful when dealing with time-series data as:
1. Data often arrives late and requires aggregation states to be updated
2. Historical data needs to be backfilled while streaming data is feeding into the table

When streaming source data, `foreachBatch()` can be used to perform a merges on micro-batches of data.

In [0]:
# Create functions to merge turbine and weather data into their target Delta tables
def merge_delta(incremental, target): 
  incremental.dropDuplicates(['date','window','deviceid']).createOrReplaceTempView("incremental")
  
  try:
    # MERGE records into the target table using the specified join key
    incremental._jdf.sparkSession().sql(f"""
      MERGE INTO delta.`{target}` t
      USING incremental i
      ON i.date=t.date AND i.window = t.window AND i.deviceId = t.deviceid
      WHEN MATCHED THEN UPDATE SET *
      WHEN NOT MATCHED THEN INSERT *
    """)
  except:
    # If the †arget table does not exist, create one
    incremental.write.format("delta").partitionBy("date").save(target)
    
turbine_b_to_s = (
  spark.readStream.format('delta').table("iot.turbine_raw")                        # Read data as a stream from our source Delta table
    .groupBy('deviceId','date',F.window('timestamp','5 minutes'))              # Aggregate readings to hourly intervals
    .agg(F.avg('rpm').alias('rpm'), F.avg("angle").alias("angle"))
    .writeStream                                                               # Write the resulting stream
    .foreachBatch(lambda i, b: merge_delta(i, SILVER_PATH + "turbine_agg"))    # Pass each micro-batch to a function
    .outputMode("update")                                                      # Merge works with update mode
    .option("checkpointLocation", CHECKPOINT_PATH + "turbine_agg")             # Checkpoint so we can restart streams gracefully
    .start()
)

weather_b_to_s = (
  spark.readStream.format('delta').table("iot.weather_raw")                        # Read data as a stream from our source Delta table
    .groupBy('deviceid','date',F.window('timestamp','5 minutes'))              # Aggregate readings to hourly intervals
    .agg({"temperature":"avg","humidity":"avg","windspeed":"avg","winddirection":"last"})
    .selectExpr('date','window','deviceid','`avg(temperature)` as temperature','`avg(humidity)` as humidity',
                '`avg(windspeed)` as windspeed','`last(winddirection)` as winddirection')
    .writeStream                                                               # Write the resulting stream
    .foreachBatch(lambda i, b: merge_delta(i, SILVER_PATH + "weather_agg"))    # Pass each micro-batch to a function
    .outputMode("update")                                                      # Merge works with update mode
    .option("checkpointLocation", CHECKPOINT_PATH + "weather_agg")             # Checkpoint so we can restart streams gracefully
    .start()
)

# Create the external tables once data starts to stream in
while True:
  try:
    spark.sql(f'CREATE TABLE IF NOT EXISTS iot.turbine_agg USING DELTA LOCATION "{SILVER_PATH + "turbine_agg"}"')
    spark.sql(f'CREATE TABLE IF NOT EXISTS iot.weather_agg USING DELTA LOCATION "{SILVER_PATH + "weather_agg"}"')
    break
  except:
    pass

In [0]:
%sql
-- As data gets merged in real-time to our hourly table, we can query it immediately
SELECT * FROM iot.turbine_agg t JOIN iot.weather_agg w ON (t.date=w.date AND t.window=w.window) WHERE t.deviceid='WindTurbine-1' ORDER BY t.window DESC

deviceId,date,window,rpm,angle,date,window,deviceid,temperature,humidity,windspeed,winddirection
WindTurbine-1,2021-03-05,"List(2021-03-05T00:05:00.000+0000, 2021-03-05T00:10:00.000+0000)",8.314439862957503,7.525134880087814,2021-03-05,"List(2021-03-05T00:05:00.000+0000, 2021-03-05T00:10:00.000+0000)",WeatherCapture,26.450434826744694,71.14715865823462,7.114715865823464,SW
WindTurbine-1,2021-03-05,"List(2021-03-05T00:00:00.000+0000, 2021-03-05T00:05:00.000+0000)",8.572609698117292,7.167700152519298,2021-03-05,"List(2021-03-05T00:00:00.000+0000, 2021-03-05T00:05:00.000+0000)",WeatherCapture,25.693280544143953,69.94095654961023,6.994095654961024,W


### 2b. Delta Silver (Aggregated) to Delta Gold (Enriched)
Next we perform a streaming join of weather and turbine readings to create one enriched dataset we can use for data science and model training.

In [0]:
# Read streams from Delta Silver tables and join them together on common columns (date & window)
turbine_agg = spark.readStream.format('delta').option("ignoreChanges", True).table('iot.turbine_agg')
weather_agg = spark.readStream.format('delta').option("ignoreChanges", True).table('iot.weather_agg').drop('deviceid')
turbine_enriched = turbine_agg.join(weather_agg, ['date','window'])

# Write the stream to a foreachBatch function which performs the MERGE as before
merge_gold_stream = (
  turbine_enriched
    .selectExpr('date','deviceid','window.start as window','rpm','angle','temperature','humidity','windspeed','winddirection')
    .writeStream 
    .foreachBatch(lambda i, b: merge_delta(i, GOLD_PATH + "turbine_enriched"))
    .option("checkpointLocation", CHECKPOINT_PATH + "turbine_enriched")         
    .start()
)

# Create the external tables once data starts to stream in
while True:
  try:
    spark.sql(f'CREATE TABLE IF NOT EXISTS iot.turbine_enriched USING DELTA LOCATION "{GOLD_PATH + "turbine_enriched"}"')
    break
  except:
    pass

In [0]:
%sql SELECT * FROM iot.turbine_enriched WHERE deviceid='WindTurbine-1'

date,deviceid,window,rpm,angle,temperature,humidity,windspeed,winddirection
2021-03-05,WindTurbine-1,2021-03-05T00:00:00.000+0000,8.572609698117292,7.167700152519298,25.693280544143953,69.94095654961023,6.994095654961024,W
2021-03-05,WindTurbine-1,2021-03-05T00:05:00.000+0000,8.350495095027156,7.417794319259871,26.419850977575987,71.21974109005735,7.121974109005738,W


### 2c: Stream Delta GOLD Table to Synapse
Synapse Analytics provides on-demand SQL directly on Data Lake source formats. Databricks can also directly stream data to Synapse SQL Pools for Data Warehousing workloads like BI dashboarding and reporting. 

<img src="https://sguptasa.blob.core.windows.net/random/iiot_blog/synapse_databricks_delta.png" width=800>

In [0]:
spark.conf.set("spark.databricks.sqldw.writeSemantics", "copy")                           # Use COPY INTO for faster loads to Synapse from Databricks

write_to_synapse = (
  spark.readStream.format('delta').option('ignoreChanges',True).table('turbine_enriched') # Read in Gold turbine readings from Delta as a stream
    .writeStream.format("com.databricks.spark.sqldw")                                     # Write to Synapse (SQL DW connector)
    .option("url",dbutils.secrets.get("iot","synapse_cs"))                                # SQL Pool JDBC connection (with SQL Auth) string
    .option("tempDir", SYNAPSE_PATH)                                                      # Temporary ADLS path to stage the data (with forwarded permissions)
    .option("forwardSparkAzureStorageCredentials", "true")
    .option("dbTable", "turbine_enriched")                                                # Table in Synapse to write to
    .option("checkpointLocation", CHECKPOINT_PATH+"synapse")                              # Checkpoint for resilient streaming
    .start()
)

### 2d. Backfill Historical Data
In order to train a model, we will need to backfill our streaming data with historical data. The cell below generates 1 year of historical hourly turbine and weather data and inserts it into our Gold Delta table.

In [0]:
import pandas as pd
import numpy as np

# Function to simulate generating time-series data given a baseline, slope, and some seasonality
def generate_series(time_index, baseline, slope=0.01, period=365*24*12):
  rnd = np.random.RandomState(time_index)
  season_time = (time_index % period) / period
  seasonal_pattern = np.where(season_time < 0.4, np.cos(season_time * 2 * np.pi), 1 / np.exp(3 * season_time))
  return baseline * (1 + 0.1 * seasonal_pattern + 0.1 * rnd.randn(len(time_index)))
  
# Get start and end dates for our historical data
dates = spark.sql('select max(date)-interval 365 days as start, max(date) as end from iot.turbine_enriched').toPandas()
  
# Get the baseline readings for each sensor for backfilling data
turbine_enriched_pd = spark.table('iot.turbine_enriched').toPandas()
baselines = turbine_enriched_pd.min()[3:8]
devices = turbine_enriched_pd['deviceid'].unique()

# Iterate through each device to generate historical data for that device
print("---Generating Historical Enriched Turbine Readings---")
for deviceid in devices:
  print(f'Backfilling device {deviceid}')
  windows = pd.date_range(start=dates['start'][0], end=dates['end'][0], freq='5T') # Generate a list of hourly timestamps from start to end date
  historical_values = pd.DataFrame({
    'date': windows.date,
    'window': windows, 
    'winddirection': np.random.choice(['N','NW','W','SW','S','SE','E','NE'], size=len(windows)),
    'deviceId': deviceid
  })
  time_index = historical_values.index.to_numpy()                                 # Generate a time index

  for sensor in baselines.keys():
    historical_values[sensor] = generate_series(time_index, baselines[sensor])    # Generate time-series data from this sensor

  # Write dataframe to enriched_readings Delta table
  spark.createDataFrame(historical_values).write.format("delta").mode("append").saveAsTable("iot.turbine_enriched")
  
# Create power readings based on weather and operating conditions
print("---Generating Historical Turbine Power Readings---")
spark.sql(f'CREATE TABLE iot.turbine_power USING DELTA PARTITIONED BY (date) LOCATION "{GOLD_PATH + "turbine_power"}" AS SELECT date, window, deviceId, 0.1 * (temperature/humidity) * (3.1416 * 25) * windspeed * rpm AS power FROM iot.turbine_enriched')

# Create a maintenance records based on peak power usage
print("---Generating Historical Turbine Maintenance Records---")
spark.sql(f'CREATE TABLE iot.turbine_maintenance USING DELTA LOCATION "{GOLD_PATH + "turbine_maintenance"}" AS SELECT DISTINCT deviceid, FIRST(date) OVER (PARTITION BY deviceid, year(date), month(date) ORDER BY power) AS date, True AS maintenance FROM iot.turbine_power')

---Generating Historical Enriched Turbine Readings---
Backfilling device WindTurbine-0
Backfilling device WindTurbine-1
Backfilling device WindTurbine-2
Backfilling device WindTurbine-3
Backfilling device WindTurbine-4
Backfilling device WindTurbine-5
Backfilling device WindTurbine-6
Backfilling device WindTurbine-7
Backfilling device WindTurbine-8
Backfilling device WindTurbine-9
---Generating Historical Turbine Power Readings---
---Generating Historical Turbine Maintenance Records---
Out[9]: DataFrame[]

In [0]:
%sql
-- Optimize all 3 tables for querying and model training performance
OPTIMIZE iot.turbine_enriched WHERE date<current_date() ZORDER BY deviceid, window;
OPTIMIZE iot.turbine_power ZORDER BY deviceid, window;
OPTIMIZE iot.turbine_maintenance ZORDER BY deviceid;

path,metrics
null,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 1218), 0, List(0, 0), 0, null), 0)"


Our Delta Gold tables are now ready for predictive analytics! We now have hourly weather, turbine operating and power measurements, and daily maintenance logs going back one year. We can see that there is significant correlation between most of the variables.

In [0]:
%sql
-- Query all 3 tables
CREATE OR REPLACE VIEW iot.gold_readings AS
SELECT r.*, 
  p.power, 
  ifnull(m.maintenance,False) as maintenance
FROM iot.turbine_enriched r 
  JOIN iot.turbine_power p ON (r.date=p.date AND r.window=p.window AND r.deviceid=p.deviceid)
  LEFT JOIN iot.turbine_maintenance m ON (r.date=m.date AND r.deviceid=m.deviceid);
  
SELECT * FROM iot.gold_readings ORDER BY deviceid, window

date,deviceid,window,rpm,angle,temperature,humidity,windspeed,winddirection,power,maintenance
2020-03-05,WindTurbine-0,2020-03-05T00:00:00.000+0000,7.702966694267955,5.672139894080909,27.43929217651895,74.10721845991635,7.410721845991635,NE,166.00524927526,false
2020-03-05,WindTurbine-0,2020-03-05T00:05:00.000+0000,7.257131030735379,5.3438453091894855,25.851148864996407,69.81800857157342,6.981800857157341,S,147.34510413737854,false
2020-03-05,WindTurbine-0,2020-03-05T00:10:00.000+0000,7.985286873127906,5.880028570349912,28.44496520907014,76.82331006472438,7.682331006472438,W,178.39670420601195,false
2020-03-05,WindTurbine-0,2020-03-05T00:15:00.000+0000,7.69464789691621,5.6660142824558815,27.40965918962552,74.02718657646965,7.402718657646964,NW,165.64688907490947,false
2020-03-05,WindTurbine-0,2020-03-05T00:20:00.000+0000,7.759449948248027,5.713731780812846,27.64049524159166,74.65062167171304,7.465062167171303,NW,168.44869592270456,false
2020-03-05,WindTurbine-0,2020-03-05T00:25:00.000+0000,7.8313531366666425,5.766678257115844,27.896626765165536,75.34237401903502,7.534237401903501,NE,171.5850307172491,false
2020-03-05,WindTurbine-0,2020-03-05T00:30:00.000+0000,7.881756211283461,5.8037929561228045,28.076171185633424,75.82728221219384,7.582728221219384,E,173.8008020703204,false
2020-03-05,WindTurbine-0,2020-03-05T00:35:00.000+0000,7.966215764915638,5.865985410356032,28.37703064163188,76.63983441973544,7.663983441973542,SW,177.5455988737502,false
2020-03-05,WindTurbine-0,2020-03-05T00:40:00.000+0000,8.405733456270893,6.1896277069202394,29.94266824964857,80.86826157813137,8.086826157813135,NE,197.67739533204198,false
2020-03-05,WindTurbine-0,2020-03-05T00:45:00.000+0000,8.677908705457916,6.39004608471797,30.912203297906927,83.48675279735139,8.348675279735138,NE,210.6861246227178,false


#### Benefits of Delta Lake on Time-Series Data
A key component of this architecture is the Azure Data Lake Store (ADLS), which enables the write-once, access-often analytics pattern in Azure. However, Data Lakes alone do not solve challenges that come with time-series streaming data. The Delta storage format provides a layer of resiliency and performance on all data sources stored in ADLS. Specifically for time-series data, Delta provides the following advantages over other storage formats on ADLS:

|**Required Capability**|**Other formats on ADLS**|**Delta Format on ADLS**|
|--------------------|-----------------------------|---------------------------|
|**Unified batch & streaming**|Data Lakes are often used in conjunction with a streaming store like CosmosDB, resulting in a complex architecture|ACID-compliant transactions enable data engineers to perform streaming ingest and historically batch loads into the same locations on ADLS|
|**Schema enforcement and evolution**|Data Lakes do not enforce schema, requiring all data to be pushed into a relational database for reliability|Schema is enforced by default. As new IoT devices are added to the data stream, schemas can be evolved safely so downstream applications don’t fail|
|**Efficient Upserts**|Data Lakes do not support in-line updates and merges, requiring deletion and insertions of entire partitions to perform updates|MERGE commands are effective for situations handling delayed IoT readings, modified dimension tables used for real-time enrichment, or if data needs to be reprocessed|
|**File Compaction**|Streaming time-series data into Data Lakes generate hundreds or even thousands of tiny files|Auto-compaction in Delta optimizes the file sizes to increase throughput and parallelism|
|**Multi-dimensional clustering**|Data Lakes provide push-down filtering on partitions only|ZORDERing time-series on fields like timestamp or sensor ID allows Databricks to filter and join on those columns up to 100x faster than simple partitioning techniques|